In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
df = pd.read_csv('train.csv')

In [13]:
df.shape

(404290, 6)

In [14]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [15]:
new_df = df.sample(30000)

In [16]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [17]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [18]:
new_df.duplicated().sum()

0

In [19]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
287303,What universities does Aaron's recruit new gra...,What universities does Express recruit new gra...
323002,What would you do if you were the leader of a ...,If you were elected the leader of whatever cou...
206530,What are the safety precautions on handling sh...,What are the safety precautions on handling sh...
74771,How can I fight against laziness?,What are good ways to fight against laziness?
224662,What is the translation of this song: Sheyda S...,Can someone please translate this song from He...


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [21]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [22]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [23]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
287303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323002,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,1
206530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
74771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
224662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.drop(columns=['is_duplicate']), temp_df['is_duplicate'],test_size=0.2,random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7428333333333333

In [27]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train.values, y_train.values)
y_pred = xgb.predict(X_test.values)
accuracy_score(y_test,y_pred)

0.7196666666666667